<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_MP1_Food_Vision_Big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Milestone Project: Food Vision Big

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras import mixed_precision, Model, Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.layers import RandomFlip, RandomHeight, RandomRotation, RandomWidth, RandomZoom
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report

## Setup

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-darkgrid')

### Constantes

In [ ]:
SEMENTE = 2008193

FORMATO_ENTRADA = (224, 224, 3)
ATIVACAO = 'softmax'

PERDA = 'sparse_categorical_crossentropy'
METRICAS = ['accuracy']

# OTIMIZADOR = 'Adam'
APRENDIZADO = 0.001

ITERACOES = 5

### Funções 

In [ ]:
def preprocessar_imagem(imagem, rotulo, tamanho=224, escalonar=False):
    """
    Redimensiona imagem para (tamanho, tamanho) e converte o dtype para float32.

    Args:
        imagem (tensor): Tensor no formato [lote, altura, largura, canais] ou [altura, largura, canais].
        rotulo (int): Rótulo (não será processado).
        tamanho (int): Tamanho em que a imagem será redimensionada.
        escalonar (bool): A imagem será escalonada ou não.
    
    Return:
        [lote, altura, largura, canais] ou [altura, largura, canais] (float32), rotulo
    """

    imagem = tf.image.resize(imagem, [tamanho, tamanho])

    if escalonar:
        imagem = tf.divide(imagem, 255.)

    return tf.cast(imagem, tf.float32), rotulo

## TensorFlow Dataset: Food101

https://www.tensorflow.org/datasets/overview

In [ ]:
(dados_treino, dados_teste), dados_info = tfds.load(name='food101',
                                                    split=['train', 'validation'],
                                                    shuffle_files=True,
                                                    as_supervised=True, # Dados em formato tuple (data, label)
                                                    with_info=True)

In [ ]:
rotulos = dados_info.features['label'].names

### Exemplo

In [ ]:
# exemplo = dados_treino.take(1) # Seleção aleatória
# exemplo

In [ ]:
# for imagem, rotulo in exemplo:
#     print(f"""
#     Formato da imagem: {imagem.shape}
#     Tipo de dado da imagem: {imagem.dtype}
#     Alcance dos valores da imagem: {tf.reduce_min(imagem)}, {tf.reduce_max(imagem)}
#     Formato do rótulo: {rotulo.shape}
#     Tipo de dado do rótulo: {rotulo.dtype}
#     Rótulo: {rotulo} ({rotulos[rotulo]})
#     """)

# plt.imshow(imagem)
# plt.title(rotulos[rotulo])
# plt.axis(False);

### Preparar e lotear datasets

https://www.tensorflow.org/guide/data_performance

**Best practice summary**

Here is a summary of the best practices for designing performant TensorFlow input pipelines:

* **Use the `prefetch` transformation** to overlap the work of a producer and consumer
* **Parallelize the data reading transformation** using the `interleave` transformation
* **Parallelize the `map` transformation** by setting the `num_parallel_calls` argument
* **Use the `cache` transformation** to cache data in memory during the first epoch
* **Vectorize user-defined functions** passed in to the `map` transformation
* **Reduce memory usage** when applying the `interleave`, `prefetch`, and `shuffle` transformations

In [ ]:
# Mapear dados de treino (função de pré-processamento e paralelização).
dados_treino = dados_treino.map(map_func=preprocessar_imagem, num_parallel_calls=tf.data.AUTOTUNE)

# Embaralhar dados de treino.
dados_treino = dados_treino.shuffle(buffer_size=1000)

# Lotear e pré-buscar dados de treino.
dados_treino = dados_treino.batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Mapear, lotear e pré-bucar dados de teste.
dados_teste = dados_teste.map(map_func=preprocessar_imagem, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

dados_treino, dados_teste

## Precisão mista

https://www.tensorflow.org/guide/mixed_precision

In [ ]:
# Bugado para EfficientNetBX em TensorFlow 2.5+
# `x` and `y` must have the same dtype, got tf.float16 != tf.float32.

# mixed_precision.set_global_policy("mixed_float16")

## Modelo

In [ ]:
modelo_base = EfficientNetB0(include_top=False)
modelo_base.trainable = False

entradas = Input(shape=FORMATO_ENTRADA, name='camada_entrada')

expansao_dados = Sequential(name='expansao_dados')

expansao_dados.add(RandomFlip('horizontal'))
expansao_dados.add(RandomHeight(0.2))
expansao_dados.add(RandomRotation(0.2))
expansao_dados.add(RandomWidth(0.2))
expansao_dados.add(RandomZoom(0.2))

# Há um bug na versão 2.8 do TensorFlow que faz necessário forçar o treinamento para que a expansão dos dados funcione.
expandidos = expansao_dados(entradas, training=True)

camadas = modelo_base(expandidos, training=False)
camadas = GlobalAveragePooling2D(name='agrupamento_media_global')(camadas)

saidas = Dense(len(rotulos), activation=ATIVACAO, name='camada_saida')(camadas)

modelo = Model(inputs=entradas, outputs=saidas)

modelo.compile(loss=PERDA,
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

### Verificar uso de precisão mista

In [ ]:
# for camada in modelo.layers:
#     print(camada.name, camada.trainable, camada.dtype, camada.dtype_policy)

### Ajustar

In [ ]:
historico = modelo.fit(dados_treino,
                       epochs=ITERACOES,
                       steps_per_epoch=len(dados_treino),
                    #    validation_data=dados_teste,
                    #    validation_steps=len(dados_teste),
                       verbose=1)

### Avaliar

In [ ]:
modelo.evaluate(dados_teste)

In [ ]:
classes_verdadeiras = []

for imagem, classe in dados_teste.unbatch():
    classes_verdadeiras.append(classe.numpy().argmax())

previsoes = modelo.predict(dados_teste, verbose=1)

classes_previstas = previsoes.argmax(axis=1)

print(classification_report(y_true=classes_verdadeiras,
                            y_pred=classes_previstas,
                            target_names=rotulos))